In [1]:
#imports
from data_loader import DataLoader
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np

In [2]:
# about dataset:

# data is avalable here: https://archive.ics.uci.edu/ml/datasets/Bank+Marketing 
# This dataset is public available for research. The details are described in 
# [Moro et al., 2014] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to 
# Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier,
# 62:22-31, June 2014




# Input variables:
## bank client data:
1 - age (numeric)

2 - job : type of job (categorical: 'admin.','blue-collar', 'entrepreneur', 'housemaid', 'management', 'retired', 'self-employed', 'services', 'student', 'technician', 'unemployed', 'unknown')

3 - marital : marital status (categorical: 'divorced', 'married', 'single', 'unknown'; note: 'divorced' means divorced or widowed)

4 - education (categorical:'basic.4y', 'basic.6y', 'basic.9y', 'high.school', 'illiterate' ,'professional.course', 'university.degree', 'unknown')

5 - default: has credit in default? (categorical: 'no','yes','unknown')

6 - housing: has housing loan? (categorical: 'no','yes','unknown')

7 - loan: has personal loan? (categorical: 'no','yes','unknown')

## related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')

9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov, 'dec')

10 - day_of_week: last contact day of the week (categorical: 'mon', 'tue', 'wed', 'thu', 'fri')

11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.

## other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)

13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)

14 - previous: number of contacts performed before this campaign and for this client (numeric)

15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')

## social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)

17 - cons.price.idx: consumer price index - monthly indicator (numeric)

18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)

19 - euribor3m: euribor 3 month rate - daily indicator (numeric)

20 - nr.employed: number of employees - quarterly indicator (numeric)

# Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')


In [3]:
# IDEJE 
# 14 - previous: --> convertat u 0 ili 1 je li bilo poziva prije ili ne
# kad je 0 u previous onda je unkwon u outcome -> moze utjecat na vizualizaciju /korelaciju

# plot kategoricki po labelama triba normalizirat jer je previse zastupljen broj negativa + odredenih poslova
# pa se ni ne vidi ako je neka kategorija problematicna

# 11-> duration -> tribalo bi izbacit -> pogledaj opis na kraju file-a

In [4]:
dl = DataLoader("./config.json") # TODO load bank full later
inputs, labels = dl.load_csv_data() #  TODO load all
print("number of null elements:", inputs.isnull().values.sum())
inputs.head()


loading data from: C:\Users\dujem\PycharmProjects\bank-marketing\data\bank.csv
number of null elements: 0


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown


In [5]:
inputs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        4521 non-null   int64 
 1   job        4521 non-null   object
 2   marital    4521 non-null   object
 3   education  4521 non-null   object
 4   default    4521 non-null   object
 5   balance    4521 non-null   int64 
 6   housing    4521 non-null   object
 7   loan       4521 non-null   object
 8   contact    4521 non-null   object
 9   day        4521 non-null   int64 
 10  month      4521 non-null   object
 11  duration   4521 non-null   int64 
 12  campaign   4521 non-null   int64 
 13  pdays      4521 non-null   int64 
 14  previous   4521 non-null   int64 
 15  poutcome   4521 non-null   object
dtypes: int64(7), object(9)
memory usage: 565.2+ KB


In [6]:
numeric_columns = ['age', 'balance', 'day', 'campaign', 'duration', 'pdays', 'previous']

all_columns = inputs.columns
categorical_columns = list(set(all_columns) - set(numeric_columns))
print(categorical_columns)

['education', 'default', 'month', 'loan', 'job', 'marital', 'contact', 'poutcome', 'housing']


In [7]:
numerical_data = inputs[numeric_columns]
numerical_data = numerical_data.join(labels)
numerical_data.head()

,age,balance,day,campaign,duration,pdays,previous,y
0,30,1787,19,1,79,-1,0,no
1,33,4789,11,1,220,339,4,no
2,35,1350,16,1,185,330,1,no
3,30,1476,3,4,199,-1,0,no
4,59,0,5,1,226,-1,0,no


In [8]:
categorical_data = inputs[categorical_columns]
categorical_data = categorical_data.join(labels)
categorical_data.head()

,education,default,month,loan,job,marital,contact,poutcome,housing,y
0,primary,no,oct,no,unemployed,married,cellular,unknown,no,no
1,secondary,no,may,yes,services,married,cellular,failure,yes,no
2,tertiary,no,apr,no,management,single,cellular,failure,yes,no
3,tertiary,no,jun,yes,management,married,unknown,unknown,yes,no
4,secondary,no,may,no,blue-collar,married,unknown,unknown,yes,no


# Visualization functions

In [9]:
FONTSIZE = 20
TARGET_COLUMN = "y"

## for numerical data:

In [10]:
# numerical data visualization with seaborn

def plot_distribution_of_numerical_data(column_data, histogram=True, kde=True, rug=False, nbins=None):
    xlabel = column_data.name
    plt.figure(figsize=(16,8))
    plt.xlabel(xlabel, fontsize=FONTSIZE)
    plt.ylabel('Probability Density', fontsize=FONTSIZE) 
    if nbins is not None:
        sns.distplot(column_data, hist=histogram, kde=kde, rug=rug, bins=bins);
    else:       
        sns.distplot(column_data, kde=kde, rug=rug);
        
    plt.show()
    
def plot_distribution_of_numerical_data_with_target(data_frame, column_name, xlabel="", histogram=True, kde=True, rug=False, nbins=None):
    class1_value = "no"
    class2_value = "yes"
    target = 'y'
    fontsize = 20
    
    class1 = data_frame.loc[data_frame[target] == class1_value][column_name]
    class2 = data_frame.loc[data_frame[target] == class2_value][column_name]
    
    plt.figure(figsize=(16,8))
    sns.distplot(class1, kde=kde, hist=histogram, rug=rug, label=class1_value)
    sns.distplot(class2, kde=kde, hist=histogram, rug=rug, label=class2_value)
    
    plt.xlabel(xlabel, fontsize=FONTSIZE) 
    plt.legend()
    plt.ylabel('Probability Density', fontsize=FONTSIZE) 
    plt.show()

## for categorical data:

In [11]:
# categorical data visualization with seaborn

def plot_categorical_histogram(column_name, data_frame,  xlabel="", ylabel="Frequency", title=""):
    att_count = data_frame[column_name].value_counts()
    plt.figure(figsize=(16,8))
    sns.set(style="darkgrid")
    sns.barplot(att_count.index, att_count.values, alpha=0.9)
    plt.title(title)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)
    plt.show()

"""
def plot_categorical_histogram_with_target1(column_name, data_frame,  xlabel="", ylabel="Frequency", title=""):
    #coloring not setup, labels too crunched
    att_count = data_frame[[column_name, 'y']].groupby(['y', column_name]).size()
    #att_count = data_frame[column_name].value_counts()
    plt.figure(figsize=(16,8))
    sns.set(style="darkgrid")
    sns.barplot(att_count.index, att_count.values,alpha=0.9)
    plt.title(title)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)
    plt.show()
"""    
    
def autolabel(rects, ax):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')
        
def plot_categorical_histogram_with_target(column_name, data_frame, ylabel="Frequency", title="", normalize=True):
    class1_name = "no"
    class2_name = "yes"
    target = 'y'
    class1_count = 0
    class2_count = 0
    
    att_count = data_frame[[column_name, target]].groupby([target, column_name]).size()
    
    class1 = att_count[class1_name]
    class2 = att_count[class2_name]
    for value1, value2 in zip(class1,class2):
        class1_count += value1
        class2_count += value2
        
    fig, ax = plt.subplots(figsize=(16,8))
    ax.set_ylabel('Frequency')
    if normalize:
        class1 = [int(val/class1_count * 100) for val in class1]
        class2 = [int(val/class2_count * 100) for val in class2]
        ax.set_ylabel('Frequency (normalized)')
        
    columns = data_frame[column_name].unique()

    x = np.arange(len(columns))  # the label locations
    width = 0.35  # the width of the bars

    rects1 = ax.bar(x - width/2, class1, width, label=class1_name)
    rects2 = ax.bar(x + width/2, class2, width, label=class2_name)

    ax.set_xticks(x)
    ax.set_xticklabels(columns)
    ax.legend()
    
    autolabel(rects1, ax)
    autolabel(rects2, ax)
    ax.title.set_text(title)
    fig.tight_layout()

    plt.show()

## correlation:

In [12]:
# correlation visialization
def plot_correlation(data_frame):
    """
    This is basic implementation that works with numerical data only.
    """
    sns.set(style="white")
    
    # Compute the correlation matrix
    corr = data_frame.corr()

    # Generate a mask for the upper triangle
    mask = np.triu(np.ones_like(corr, dtype=np.bool))

    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(16, 8))

    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(220, 10, as_cmap=True)

    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5})
    plt.show()
    
def plot_correlation_for_categorical_data(data_frame, theilu=True, ):
    """
    This function uses the code from: https://github.com/shakedzy/dython
    To use this part run: pip install dython
    This function can also be used to plot correlation of continuios or mixed
    cases.
    """
    try:
        from dython import nominal
    except ImportError as e:
        raise ModuleNotFoundError("dython module not found")
    
    theilu = nominal.associations(dataset=data_frame, theil_u=True)

In [13]:
full_data = inputs.join(labels)
plot_correlation_for_categorical_data(full_data)

ModuleNotFoundError: dython module not found

In [ ]:
plot_correlation_for_categorical_data(categorical_data)

In [ ]:
plot_correlation_for_categorical_data(numerical_data)

In [ ]:
plot_correlation(numerical_data)

# Ploting data distribution

In [ ]:
# correlation visialization
def plot_correlation(data_frame):
    """
    This is basic implementation that works with numerical data only.
    """
    sns.set(style="white")
    
    # Compute the correlation matrix
    corr = data_frame.corr()

    # Generate a mask for the upper triangle
    mask = np.triu(np.ones_like(corr, dtype=np.bool))

    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(16, 8))

    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(220, 10, as_cmap=True)

    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5})
    plt.show()
    
def plot_correlation_for_categorical_data(data_frame, theilu=True, ):
    """
    This function uses the code from: https://github.com/shakedzy/dython
    To use this part run: pip install dython
    This function can also be used to plot correlation of continuios or mixed
    cases.
    """
    try:
        from dython import nominal
    except ImportError as e:
        raise ModuleNotFoundError("dython module not found")
    
    theilu = nominal.associations(dataset=data_frame, theil_u=True)

In [ ]:
full_data = inputs.join(labels)
plot_correlation_for_categorical_data(full_data)

In [ ]:
plot_correlation_for_categorical_data(categorical_data)

In [ ]:
plot_correlation_for_categorical_data(numerical_data)

In [ ]:
plot_correlation(numerical_data)

# Ploting data distribution

## for numerical data:

In [ ]:
for column in numerical_data:
    if column != TARGET_COLUMN:
        plot_distribution_of_numerical_data(numerical_data[column])

In [ ]:
for column in numerical_data:
    if column != TARGET_COLUMN:
        plot_distribution_of_numerical_data_with_target(numerical_data, column_name=column, 
                                                    xlabel=column)

## for categorical data:

In [ ]:
for column in categorical_columns:
    plot_categorical_histogram(column_name=column, title=column, data_frame=categorical_data)

In [ ]:
for column in categorical_columns:
    plot_categorical_histogram_with_target(column_name=column, title=column, data_frame=categorical_data, normalize=True)